In [1]:
from dotenv import load_dotenv

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
#from langchain.llms import CTransformers
from langchain_community.llms import CTransformers
import os
import pinecone
from pinecone import Pinecone

/Users/mtm007/Downloads/LLM/Medchatbot/RAG_MedChatBot/Rchat/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()
key= os.getenv("PINECONE_API_KEY")

In [4]:
PINECONE_API_KEY = key
PINECONE_API_ENV = 'gcp-starter'

In [5]:
#extract data func
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [10]:
#path = '../'
extracted_data = load_pdf_data("../data/book2")

In [11]:
#create chunks of text
def text_chunk_splitter(extracted_data):
    text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunk = text_split.split_documents(extracted_data)

    return text_chunk

In [12]:
text_chunks = text_chunk_splitter(extracted_data)
print("chunk length:  ", len(text_chunks))

chunk length:   14


In [ ]:
#download embedding model
def download_embedding_model():
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embedding


In [ ]:
embedding = download_embedding_model()

In [ ]:
query_res = embedding.embed_query("Hi today is March 12 Tuesday")
print("Length: ", len(query_res))

In [ ]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),  
   environment=os.getenv("PINECONE_ENV"),  
)
index_name = "medical-chatbot"

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding)
query = "what is allergy?"
docs = docsearch.similarity_search(query,k=3)

print("Results", docs)

In [ ]:
prompt_template = """ 
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.


Context : {context}
Question: {question}

Only return helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q3_0.bin")

In [ ]:
llm = CTransformers(model= "model/llama-2-7b-chat.ggmlv3.q3_K_M.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512,
                            'temperature': 0.7})

In [ ]:
QA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(search_kwargs={'k':3}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input: input(f"Input prompt: ")
    result = QA({"query": user_input})
    print("Response: ", result["result"])